# Navigation

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below.  Please run the next code cell without making any changes.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# please do not modify the line below
env = UnityEnvironment(file_name="/data/Banana_Linux_NoVis/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [ 1.          0.          0.          0.          0.84408134  0.          0.
  1.          0.          0.0748472   0.          1.          0.          0.
  0.25755     1.          0.          0.          0.          0.74177343
  0.          1.          0.          0.          0.25854847  0.          0.
  1.          0.          0.09355672  0.          1.          0.          0.
  0.31969345  0.          0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agent while it is training**, and you should set `train_mode=True` to restart the environment.

In [5]:
env_info = env.reset(train_mode=True)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 0.0


When finished, you can close the environment.

In [6]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agent while it is training.  However, **_after training the agent_**, you can download the saved model weights to watch the agent on your own machine! 

In [7]:
import random
from collections import namedtuple, deque
import time
import matplotlib.pyplot as plt
import torch
%matplotlib inline

# Add-on : Set plotting options
plt.style.use('ggplot')
np.set_printoptions(precision=3, linewidth=120)
# Add-on : Hide Matplotlib deprecate warnings
import warnings
warnings.filterwarnings("ignore")


# High resolution plot outputs for retina display
%config InlineBackend.figure_format = 'retina'

# Import Agent Class and Initialize 
# Import DQN Agent
'''
from ddqn_agent import Agent
agent = Agent(state_size=37, action_size=4, seed=42)
save_trained_agent_path = 'checkpoint_dqn'
'''
# Import DDQN Agent
'''
from ddqn_agent import DDQN_Agent
agent = DDQN_Agent(state_size=37, action_size=4, seed=42)
save_trained_agent_path = 'checkpoint_ddqn'
'''
# Import Prioritized DQN Agent
from dqn_per_agent import DQN_PER_Agent
agent = DQN_PER_Agent(state_size=37, action_size=4, seed=42)
save_trained_agent_path = 'checkpoint_dqn_per'

In [8]:
# Deep Q-Learning function
def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        state = env_info.vector_observations[0]            # get the current state

        score = 0
        for t in range(max_t):           
            #action = np.random.randint(action_size)        # select an action
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        
        tr_save_check = 13 # Threshold score for model parameter save 
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=tr_save_check:
            #print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), save_trained_agent_path + '_{}.pth'.format(tr_save_check))
            #break
    torch.save(agent.qnetwork_local.state_dict(), save_trained_agent_path + '_final.pth') # final model parameter save
    return scores


# Train the agent using DQN
'''
start_time = time.time() # Monitor Training Time  
scores = dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995)
print("\nTotal Training time = {:.1f} min".format((time.time()-start_time)/60))
'''

'\nstart_time = time.time() # Monitor Training Time  \nscores = dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995)\nprint("\nTotal Training time = {:.1f} min".format((time.time()-start_time)/60))\n'

In [9]:
# plot the scores
'''
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.title('Score (Rewards)')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.grid(True)      
plt.show()
'''

"\nfig = plt.figure(figsize=(8,8))\nax = fig.add_subplot(111)\nplt.plot(np.arange(len(scores)), scores)\nplt.title('Score (Rewards)')\nplt.ylabel('Score')\nplt.xlabel('Episode #')\nplt.grid(True)      \nplt.show()\n"

### 5. Evaluate the trained Agent

In [10]:
agent.qnetwork_local.load_state_dict(torch.load(save_trained_agent_path + '_13.pth')) # load trained agent parameters

num_episodes = 100
scores = []
final_eps=0.01
for i_episode in range(1,num_episodes+1):
    env_info = env.reset(train_mode=False)[brain_name] # reset the environment
    state = env_info.vector_observations[0]            # get the current state
    score = 0                                          # initialize the score
    while True:
        action = agent.act(state, eps=final_eps)        # select an action
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        #agent.step(state, action, reward, next_state, done) #do the learning

        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            print('Episode {}: {}'. format(i_episode, score))
            scores.append(score)
            break

print('All the scores{}'.format(scores))
print("Mean Score: {}".format(np.mean(scores)))

Episode 1: 21.0
Episode 2: 17.0
Episode 3: 18.0
Episode 4: 20.0
Episode 5: 19.0
Episode 6: 14.0
Episode 7: 18.0
Episode 8: 16.0
Episode 9: 14.0
Episode 10: 14.0
Episode 11: 6.0
Episode 12: 22.0
Episode 13: 22.0
Episode 14: 20.0
Episode 15: 14.0
Episode 16: 13.0
Episode 17: 18.0
Episode 18: 20.0
Episode 19: 18.0
Episode 20: 15.0
Episode 21: 15.0
Episode 22: 4.0
Episode 23: 20.0
Episode 24: 15.0
Episode 25: 11.0
Episode 26: 16.0
Episode 27: 15.0
Episode 28: 12.0
Episode 29: 19.0
Episode 30: 15.0
Episode 31: 20.0
Episode 32: 15.0
Episode 33: 16.0
Episode 34: 18.0
Episode 35: 16.0
Episode 36: 18.0
Episode 37: 17.0
Episode 38: 20.0
Episode 39: 19.0
Episode 40: 3.0
Episode 41: 11.0
Episode 42: 16.0
Episode 43: 7.0
Episode 44: 21.0
Episode 45: 16.0
Episode 46: 8.0
Episode 47: 12.0
Episode 48: 16.0
Episode 49: 7.0
Episode 50: 14.0
Episode 51: 9.0
Episode 52: 15.0
Episode 53: 12.0
Episode 54: 13.0
Episode 55: 18.0
Episode 56: 13.0
Episode 57: 13.0
Episode 58: 11.0
Episode 59: 18.0
Episode 60: 1

In [11]:
env.close()